## 5.1 Tarea

En esta práctica describo en primer término la tarea:  **El objetivo es desarrollar un prototipo de sistema que identifique la matrícula de un vehículo, bien desde una imagen o desde un vídeo**. Como alternativa, será admisible un escenario donde se combine el uso de detectores de objetos, y reconocimiento de texto.

Nos centraremos en matrículas españolas, siendo una primera subtarea recopilar o capturar imágenes o vídeos que contengan vehículos con su matrícula visible. Si necesitan cámaras, trípode, etc. hablen conmigo.

Si bien cuentan con libertad a la hora de escoger los módulos que integren en el prototipo, les propongo los siguientes apartados:

- un detector de objetos, que permita localizar vehículos
- un localizador de matrículas
- y un reconocedor de texto

Para la detección, les propongo hacer uso de YOLOv8, para el reconocimiento de texto, les muestro dos OCRs diferentes. De cara a localizar las matrículas, les sugiero dos fases:

- En una primera fase, tras detectar un coche, las zonas probables de la matrícula estarán en su parte inferior, y además se asume que se corresponde a una zona rectangular (su contorno lo es)
- En una segunda fase, se plantea realizar un entrenamiento de YOLOv8 para detectar el objeto de interés: matrículas

## Importaciones necesarias

In [ ]:
import os, re, cv2, easyocr, numpy as np, matplotlib.pyplot as plt
from IPython.display import display, Image
from ultralytics import YOLO
from abc import ABC, abstractmethod

## Detección de Vehículos

In [2]:
class IDetectorVehiculos(ABC):
    @abstractmethod
    def detectar(self, img):
        pass

class DetectorVehiculos(IDetectorVehiculos):
    def __init__(self, model):
        self.model = YOLO(model).cuda()
        
    def detectar(self, img):
        results = self.model(img, stream=False)
        boxes, class_names = [], []
        for r in results:
            boxes.extend(r.boxes)
            class_names.extend([self.model.names[int(box.cls[0])] for box in r.boxes])
        return boxes, class_names

## Procesamiento de Imagen

In [1]:
class IFiltro(ABC):
    @abstractmethod
    def aplicar(self, img):
        pass

    @abstractmethod
    def mostrar(self, img):
        pass


class FiltroCanny(IFiltro):
    def aplicar(self, img):
        # Asumimos que la imagen ya está en escala de grises
        return cv2.Canny(img, 50, 200)

    def mostrar(self, img):
        # Muestra la imagen con el rectángulo dibujado
        display(Image(data=cv2.imencode('.jpg', img)[1].tobytes()))

class FiltroSobel(IFiltro):
    def aplicar(self, img):
        # Asumimos que la imagen ya está en escala de grises
        sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        return np.hypot(sobelx, sobely).astype(np.uint8)

    def mostrar(self, img):
        # Muestra la imagen con el rectángulo dibujado
        display(Image(data=cv2.imencode('.jpg', img)[1].tobytes()))
        

class FiltroUmbralizacion(IFiltro):
    def aplicar(self, img):
        # Asumimos que la imagen ya está en escala de grises
        return cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    def mostrar(self, img):
        # Muestra la imagen con el rectángulo dibujado
        display(Image(data=cv2.imencode('.jpg', img)[1].tobytes()))





## Reconocimiento de texto (OCR)


In [3]:
    
class IOCR(ABC):
    @abstractmethod
    def localizar_matriculas(self, img):
        pass

class OCR(IOCR):
    def __init__(self, language='en'):
        self.reader = easyocr.Reader([language])        
        
    def localizar_matriculas(self, vehicle_roi):
        resultados_ocr = self.reader.readtext(vehicle_roi, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        
        # Elimina todos los espacios y convierte a mayúsculas
        texto_unificado = re.sub(r"\s+", "", "".join([res[1] for res in resultados_ocr])).upper()
        
        # Expresión regular modificada para detectar ambos formatos
        pattern = r'([0-9]{4}[A-Z]{3})|([A-Z]{3}[0-9]{4})'
        matches = re.findall(pattern, texto_unificado, re.IGNORECASE)

        # Filtra y unifica las matrículas, reorganizando si es necesario
        matricula = None
        area_matricula = None
        
        # Itera sobre los resultados para encontrar una matrícula válida
        for match in matches:
            if match[0]:
                matricula = match[0]  # Formato correcto NNNNLLL
                break
            elif match[1]:
                # Formato inverso LLLNNNN, se reorganiza a NNNNLLL
                partes = re.match(r'([A-Z]{3})([0-9]{4})', match[1])
                matricula = partes.group(2) + partes.group(1)
                break
        
        # Coordenadas generales de la matrícula (si se detectan)
        if resultados_ocr and matricula:
            # Obtiene las coordenadas mínimas y máximas de los puntos de los resultados de OCR
            x_min = min([res[0][0][0] for res in resultados_ocr])
            y_min = min([res[0][0][1] for res in resultados_ocr])
            x_max = max([res[0][2][0] for res in resultados_ocr])
            y_max = max([res[0][2][1] for res in resultados_ocr])
            area_matricula = ((int(x_min), int(y_min)), (int(x_max), int(y_max)))
        
        return matricula, area_matricula

## Procesamiento de Matrículas

In [ ]:
class ProcesadorMatriculas:
    def __init__(self, ocr, filtros=[]):
        self.ocr = ocr
        self.filtros = filtros
    
    def __aplicar_filtros(self, img):
        img_filtrada = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).copy()  # Convertimos la imagen a escala de grises
        for filtro in self.filtros:
            img_filtrada = filtro.aplicar(img_filtrada)
            # filtro.mostrar(img_filtrada)
        return img_filtrada
    
    def __aislar_matricula(self, vehicle_roi):
        # Asunción: la matrícula se encuentra en la parte inferior central de la ROI
        h, w = vehicle_roi.shape[:2]
        y1_mat = int(h * 0.2)   # Comienza al 50% de la altura de la ROI
        y2_mat = int(h * 0.8)   # Hasta el 80 $ de la altura de la ROI
        x1_mat = 0 # int(w * 0.2)   # Comienza al 25% del ancho de la ROI 
        x2_mat = w # int(w * 0.8)   # Hasta el 75% del ancho de la ROI
        matricula_roi = vehicle_roi[y1_mat:y2_mat, x1_mat:x2_mat]
        
        # Devuelve la región de interes de la matricula y las coordenadas relativas a la imagen        
        return matricula_roi, (x1_mat, y1_mat, x2_mat, y2_mat)
      
    def __ajustar_y_devolver_datos_matricula(self, matricula, area_matricula, coords_vehiculo, coords_matricula):
        x_min, y_min = coords_vehiculo[0] + coords_matricula[0] + area_matricula[0][0], coords_vehiculo[1] + coords_matricula[1] + area_matricula[0][1]
        x_max, y_max = coords_vehiculo[0] + coords_matricula[0] + area_matricula[1][0], coords_vehiculo[1] + coords_matricula[1] + area_matricula[1][1]
        return matricula, (x_min, y_min, x_max, y_max)
           
    
    def procesar_matricula(self, vehicle_roi, coords_vehiculo, class_name):
        matricula_roi, coords_matricula = self.__aislar_matricula(vehicle_roi)
        matricula_roi_filtrado = self.__aplicar_filtros(matricula_roi)
        matricula, area_matricula = self.ocr.localizar_matriculas(matricula_roi_filtrado)
        if matricula and area_matricula:
            return self.__ajustar_y_devolver_datos_matricula(matricula, area_matricula, coords_vehiculo, coords_matricula)
        return None


## Almacenamiento y visualización de imágenes

In [5]:
class Visualizador:
    def __init__(self, output_dir='./outputs'):
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

    def mostrar_y_guardar(self, img, img_name, matriculas_detectadas):
        result_image_path = os.path.join(self.output_dir, f'output_{img_name}')
        cv2.imwrite(result_image_path, img)
        display(Image(filename=result_image_path))
        for matricula in matriculas_detectadas:
            print(matricula)
    
    def mostrar_imagen(self, img, titulo='Imagen'):
        display(Image(data=cv2.imencode('.jpg', img)[1].tobytes(), title=titulo))

## Clase Principal de Detección de Matrículas

In [4]:

class DetectorDeMatriculas:
    def __init__(self, detector_vehiculos, procesador_matriculas):
        self.detector_vehiculos = detector_vehiculos
        self.procesador_matriculas = procesador_matriculas
        self.visualizador = Visualizador()
        self.__objNames = {"car": "coche", "motorbike": "motocicleta", "bus": "autobús"}

    def detectar(self, path, es_video=False):
        if es_video:
            self.__procesar_video(path)
        else:
            self.__procesar_imagen(path)
    
    def __mostrar_matricula(self, img, resultado_matricula ):
        matricula, (x_min, y_min, x_max, y_max) = resultado_matricula
        
        # Calcula el tamaño del texto para el fondo
        text_size = cv2.getTextSize(matricula, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
        text_x_start = x_min
        text_y_start = y_min - 10 - text_size[1]
        text_x_end = x_min + text_size[0] + 10
        text_y_end = y_min

        # Dibuja el fondo blanco detrás del texto
        cv2.rectangle(img, (text_x_start, text_y_start), (text_x_end, text_y_end), (255, 255, 255), cv2.FILLED)
        
        # Escribe el texto de la matrícula sobre el fondo blanco
        cv2.putText(img, matricula, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        
        # Dibuja un rectángulo alrededor del área de la matrícula
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        
        # Devuelve la matricula
        return matricula
        

    def __procesar_vehiculo(self, img, box, class_name):
        x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(img, self.__objNames[class_name], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        resultado_matricula = self.procesador_matriculas.procesar_matricula(img[y1:y2, x1:x2], (x1, y1, x2, y2), class_name)
        if resultado_matricula:
            matricula = self.__mostrar_matricula(img, resultado_matricula)
            return f"Matrícula del {self.__objNames[class_name]}: {matricula}"
        return None

    def __procesar_fotograma(self, img):
        boxes, class_names = self.detector_vehiculos.detectar(img)
        matriculas_detectadas = [self.__procesar_vehiculo(img, box, class_name) for box, class_name in zip(boxes, class_names) if class_name in ["car", "motorbike", "bus"]]
        return img, [mat for mat in matriculas_detectadas if mat]


    def __procesar_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
    
        while True:
            ret, img = cap.read()
            if not ret:
                print("Fin del video.")
                break
            
            img, matriculas_detectadas = self.__procesar_fotograma(img)
            cv2.imshow('Video', img)
            
            for matricula in matriculas_detectadas:
                print("Matricula: ", matricula)
    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
        cap.release()
        cv2.destroyAllWindows()
        
    def __procesar_imagen(self, img_path):
        img = cv2.imread(img_path)
        img, matriculas_detectadas = self.__procesar_fotograma(img)
        self.visualizador.mostrar_y_guardar(img, os.path.basename(img_path), matriculas_detectadas)
       


# Uso del detector de matriculas

In [ ]:
# Creación de instancias de las clases de procesamiento 
filtro_canny, filtro_sobel, filtro_umbralizacion = FiltroCanny(), FiltroSobel(),  FiltroUmbralizacion()
detector_vehiculos = DetectorVehiculos(model= 'yolov8n.pt')  
ocr = OCR('en')  

# Se incluyen en una lista aquellos filtros a aplicar
filtros = [
            # filtro_canny, 
            # filtro_sobel,
            filtro_umbralizacion
           ]

procesador_matriculas = ProcesadorMatriculas(ocr=ocr, filtros = filtros)

# Creación del detector de matrículas con inyección de dependencias
detector = DetectorDeMatriculas(detector_vehiculos=detector_vehiculos, 
                                procesador_matriculas=procesador_matriculas)

#                           ============================== USO ============================== 
# -------------------- Ejemplo con imagenes -------------------- 
# detector.detectar(f'./images/{14}.png')

# for i in range(1, 16):
    # detector.detectar(f'./images/{i}.png')
    
# -------------------- Ejemplo con vídeo -------------------- 
detector.detectar(f'./videos/video2.mp4', es_video=True)